# Imports!

In [ ]:
import pandas     as pd
import sqlalchemy as sa

from osgeo      import ogr
from sqlalchemy import MetaData, Table, Column, create_engine

import os

# Connections
<hr>

In [ ]:
mysql_cnx = \
    (
        create_engine("mysql+mysqldb://root:gdal@db/gdal?charset=utf8", echo=False)
    ) \
        .connect() 

gb_driver = ogr.GetDriverByName("ESRI Shapefile")

# DumpSHP
<hr>

##### `gb_*`

In [ ]:
def gb_open(gb_driver:ogr.Driver, path:str, name:str) -> ogr.DataSource:
    return gb_driver.Open(
            f'{path}/{name}',
            0
        )

def gb_get_layer(gb:ogr.DataSource) -> ogr.Layer:
    assert gb.GetLayerCount() == 1

    return gb[0]

def gb_get_dataframe(gb_layer:ogr.Layer) -> pd.DataFrame:
    def gb_get_layer_field_names(gb_layer:ogr.Layer) -> list:
        return \
            list(
                gb_layer[0] \
                    .items() \
                    .keys()
            ) + \
            [
                'coord_ul_x', 'coord_lr_x',
                'coord_ul_y', 'coord_lr_y'
            ]

    def gb_get_layer_data(gb_layer:ogr.Layer) -> list:
        return \
            [
                [*r] + get_envelope(r)
                for r
                in [*gb_layer]
            ]
    
    def get_envelope(feature:ogr.Layer) -> list:
        return \
            list(
                feature \
                    .GetGeometryRef() \
                    .GetEnvelope()
            )

    return \
        pd.DataFrame(
            data=gb_get_layer_data(gb_layer),
            columns=gb_get_layer_field_names(gb_layer)
        )

##### `dumpSHP`

In [ ]:
def dumpSHP(gb_driver, cnx, path, name):
    print('----------')
    print(name)    
    
    gb       = gb_open(gb_driver, path, name)
    gb_layer = gb_get_layer(gb)
    
    gb_layer_name      = gb_layer.GetName()
    gb_layer_geom_type = gb_layer.GetGeomType()
    
    pkl_file_path = f'output/{gb_layer_name}.pkl'
    
    if os.path.exists(pkl_file_path):
        return

    # wkbPoint = 1,
    # wkbLineString = 2,
    # https://gis.stackexchange.com/a/239295/41128    
    
    # ----------    
    print('Reading data')    
    data = gb_get_dataframe(gb_layer)    
    print('Read data')    
    # ----------
    
    # ----------
    print('Writing pickle')
    data.to_pickle(pkl_file_path)
    print('Wrote pickle')
    # ----------    
    
    # ----------
    print('Writing data')
    data \
        .to_sql(
            con=cnx,
            name=gb_layer_name,
            if_exists='replace'
        )
    print('Wrote data')
    # ----------

`dumpSHP`

# GEOSAMPA
<hr>

`../Assets/GEOSAMPA/GEOLOG/LAYERS_GEOLOG`

In [ ]:
GEOLOG_Path = '../Assets/GEOSAMPA/GEOLOG/LAYERS_GEOLOG'

for \
    file_name \
    in \
        [
            file_name
            for file_name
            in os.listdir(GEOLOG_Path)
            if file_name[-4:] == '.shp'
    
        ]:
    
    dumpSHP(gb_driver, mysql_cnx, GEOLOG_Path, file_name)
    
print('----------')    

`../Assets/GEOSAMPA/SIRGAS_SHP_logradouronbl`

In [ ]:
dumpSHP(
    gb_driver,
    mysql_cnx,
    '../Assets/GEOSAMPA/SIRGAS_SHP_logradouronbl',
    'SIRGAS_SHP_logradouronbl_line.shp'
)

print('----------')

# GEOfabrik - South America/Brazil/Sudeste
<hr>

`../Assets/sudeste-latest-free.shp`

In [ ]:
GEOLOG_Path = '../Assets/sudeste-latest-free.shp'

for \
    file_name \
    in \
        [
            file_name
            for file_name
            in sorted(os.listdir(GEOLOG_Path))
            if file_name[-4:] == '.shp'
    
        ]:

    dumpSHP(gb_driver, mysql_cnx, GEOLOG_Path, file_name)
    
print('----------')

In [ ]:
GEOLOG_Path = '../../Assets/sudeste-latest-free.shp'

for \
    file_name \
    in \
        [
            file_name
            for file_name
            in os.listdir(GEOLOG_Path)
            if file_name[-4:] == '.shp'
    
        ]:

    dumpSHP(gb_driver, mysql_cnx, GEOLOG_Path, file_name)